In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
no_requirements_responses = pd.read_csv(r'C:\Users\Will\Files\Florida\Research\NLP\Mechanical Turk Datasets\No Requirements - Drug Discontinuation 1__Batch_4794452_batch_results.csv')
greater_than_85_percent_acceptance_responses = pd.read_csv(r'C:\Users\Will\Files\Florida\Research\NLP\Mechanical Turk Datasets\greater than 85 percent satisfaction - Drug Discontinuation 4__Batch_4794448_batch_results.csv')

In [4]:
data1 = no_requirements_responses
data2 = greater_than_85_percent_acceptance_responses

In [5]:
# data1.head()
data1.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.m_1',
       'Input.url', 'Input.text_', 'Input.text', 'Answer.taskAnswers',
       'Approve', 'Reject'],
      dtype='object')

In [6]:
data1['Input.m_1'] = data1['Input.m_1'].str.split(' ').str[0]
data2['Input.m_1'] = data2['Input.m_1'].str.split(' ').str[0]

In [7]:
data1['Input.m_1'].head()

0    No
1    No
2    No
3    No
4    No
Name: Input.m_1, dtype: object

In [8]:
data1['Answer.taskAnswers'].head()

0    [{"q1a":{"no":false,"yes":true},"q1b":{"not_co...
1    [{"q1a":{"no":true,"yes":false},"q1b":{"not_co...
2    [{"q1a":{"no":true,"yes":false},"q1b":{"not_co...
3    [{"q1a":{"no":false,"yes":true},"q1b":{"not_co...
4    [{"q1a":{"no":true,"yes":false},"q1b":{"not_co...
Name: Answer.taskAnswers, dtype: object

In [9]:
data1['Answer.taskAnswers'][0]

'[{"q1a":{"no":false,"yes":true},"q1b":{"not_confident":false,"somewhat_confident":false,"very_confident":true},"q2a":{"q2a1":false,"q2a2":false,"q2a3":false,"q2a4":false,"q2a5":true,"q2a6":false,"q2a7":false,"q2a8":false},"q2b":{"not_confident":true,"somewhat_confident":false,"very_confident":false}}]'

In [10]:
data1 = data1.join(data1['Answer.taskAnswers'].apply(json.loads).apply(pd.Series))
data2 = data2.join(data2['Answer.taskAnswers'].apply(json.loads).apply(pd.Series))

In [11]:
data1.columns

Index([                      'HITId',                   'HITTypeId',
                             'Title',                 'Description',
                          'Keywords',                      'Reward',
                      'CreationTime',              'MaxAssignments',
               'RequesterAnnotation', 'AssignmentDurationInSeconds',
        'AutoApprovalDelayInSeconds',                  'Expiration',
               'NumberOfSimilarHITs',           'LifetimeInSeconds',
                      'AssignmentId',                    'WorkerId',
                  'AssignmentStatus',                  'AcceptTime',
                        'SubmitTime',            'AutoApprovalTime',
                      'ApprovalTime',               'RejectionTime',
                 'RequesterFeedback',           'WorkTimeInSeconds',
              'LifetimeApprovalRate',      'Last30DaysApprovalRate',
             'Last7DaysApprovalRate',                   'Input.m_1',
                         'Input.ur

In [12]:
data1[0].head()

0    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
1    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
2    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
3    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
4    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
Name: 0, dtype: object

In [13]:
def get_label(row):
    if row[0]['q1a']['no']:
        return 'No'
    else:
        return 'Yes'
data1['label'] = data1.apply(get_label, axis=1)
data2['label'] = data2.apply(get_label, axis=1)

data1['label'].head()

0    Yes
1     No
2     No
3    Yes
4     No
Name: label, dtype: object

In [29]:
def get_confidence(row):    
    if row[0]['q1b']['very_confident']:
        return 'very_confident'
    elif row[0]['q1b']['somewhat_confident']:
        return 'somewhat_confident'
    else:
        return 'not_confident'
data1['confidence'] = data1.apply(get_confidence, axis=1)
data2['confidence'] = data2.apply(get_confidence, axis=1)

data1['confidence'].head()

0        very_confident
1        very_confident
2        very_confident
3        very_confident
4    somewhat_confident
Name: confidence, dtype: object

In [14]:
# data1['correct_label'] = np.where(data1[0]['q1a'] data1['Answer.taskAnswers'].apply(lambda x: x[0]['correct_label']), 1, 0)
data1['correct_label'] = (data1['label']==data1['Input.m_1'])
data2['correct_label'] = (data2['label']==data2['Input.m_1'])
data1['correct_label'].head()

0    False
1     True
2     True
3    False
4     True
Name: correct_label, dtype: bool

In [16]:
print(data1['correct_label'].value_counts())
print(data2['correct_label'].value_counts())

True     396
False    252
Name: correct_label, dtype: int64
True     352
False    296
Name: correct_label, dtype: int64


In [31]:
n_data_points = len(data1)
print(n_data_points)

648


In [32]:
a = data1['correct_label'].value_counts()
b = data2['correct_label'].value_counts()

print(a[1]/n_data_points)
print(b[1]/n_data_points)

0.6111111111111112
0.5432098765432098


In [24]:
print(data1[0].head())

0    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
1    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
2    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
3    {'q1a': {'no': False, 'yes': True}, 'q1b': {'n...
4    {'q1a': {'no': True, 'yes': False}, 'q1b': {'n...
Name: 0, dtype: object


In [43]:
#"q1b":{"not_confident":false,"somewhat_confident":false,"very_confident":true}

data1_very_confident = data1.loc[data1['confidence']== 'very_confident']
a = data1_very_confident['correct_label'].value_counts()
print(len(data1_very_confident))
print(a)
print(a[1] / len(data1_very_confident))

data2_very_confident = data2.loc[data2['confidence']=='very_confident']
b = data2_very_confident['correct_label'].value_counts()
print(len(data2_very_confident))
print(b)
print(b[1] / len(data2_very_confident))

# print(data2_very_confident['correct_label'].value_counts())

384
True     259
False    125
Name: correct_label, dtype: int64
0.6744791666666666
318
True     213
False    105
Name: correct_label, dtype: int64
0.6698113207547169


In [45]:
data1_somewhat_confident = data1.loc[data1['confidence']== 'somewhat_confident']
a = data1_somewhat_confident['correct_label'].value_counts()
print(len(data1_somewhat_confident))
print(a)
print(a[1] / len(data1_somewhat_confident))
data2_somewhat_confident = data2.loc[data2['confidence']=='somewhat_confident']
b = data2_somewhat_confident['correct_label'].value_counts()
print(len(data2_somewhat_confident))
print(b)
print(b[1] / len(data2_somewhat_confident))

245
False    124
True     121
Name: correct_label, dtype: int64
0.49387755102040815
292
False    178
True     114
Name: correct_label, dtype: int64
0.3904109589041096


In [46]:
data1_not_confident = data1.loc[data1['confidence']== 'not_confident']
a = data1_not_confident['correct_label'].value_counts()
print(len(data1_not_confident))
print(a)
print(a[1] / len(data1_not_confident))
data2_not_confident = data2.loc[data2['confidence']=='not_confident']
b = data2_not_confident['correct_label'].value_counts()
print(len(data2_not_confident))
print(b)
print(b[1] / len(data2_not_confident))

19
True     16
False     3
Name: correct_label, dtype: int64
0.8421052631578947
38
True     25
False    13
Name: correct_label, dtype: int64
0.6578947368421053


In [26]:
data1['confidence']

0      Yes
1       No
2       No
3      Yes
4       No
      ... 
643     No
644    Yes
645     No
646     No
647     No
Name: confidence, Length: 648, dtype: object

In [ ]:
# Exclude labels which had a low confidence

# Exclude labels which all workers didn't agree on


In [97]:
print(data1['WorkerId'].value_counts())
print(data2['WorkerId'].value_counts())

A29HILIOBLRW1S    47
A2EYD5BP3AKLIE    37
A64F60OM8VDDQ     33
AFA73M8WJKEN0     29
ACU8RTAZQU1GP     28
                  ..
A380X909I6GY7D     1
A2ZLJQWCM8KU36     1
A1VK4OIA44TUL7     1
A2YVWC06HHKM7V     1
A3N6IUYI1UUMYY     1
Name: WorkerId, Length: 111, dtype: int64
A3UBH7E0BEB23K    73
AXJCV4FWC4BGQ     36
APXN6ZCXRGB5M     33
A23LU9N9IT220U    32
A19940RY4CF491    23
                  ..
A14CCWIFLDPZLA     1
A1UCL7A73YKS5C     1
A3AGJSGD009WMM     1
A25XIKDZOJCT8W     1
A1EDN55K4IF2GD     1
Name: WorkerId, Length: 133, dtype: int64


In [98]:
print(set(data1['WorkerId'].unique()).intersection(set(data2['WorkerId'].unique())))
print(len(set(data1['WorkerId'].unique()).intersection(set(data2['WorkerId'].unique()))))
# print(data2['WorkerId'].value_counts())

{'A38AQF5BPHJS4C', 'A3OTX62IHUFV3D', 'A1K0LRUQUN5I0J', 'A2N8CRZSSKEDH3', 'A5J3EJX1ADIS', 'A23LU9N9IT220U', 'A2EYD5BP3AKLIE', 'A2VUF0V7HT51Q3', 'ADVCIFLB5A9B', 'A2WYZLG8J526TQ', 'AXJCV4FWC4BGQ', 'A2V40CL511Z37G', 'A38ZFP228C5112', 'APGOZK7LO6YQF', 'A2ZLJQWCM8KU36', 'A3T467USWU7ZOY', 'A1XHSOHTQ81V4O', 'A1CZZ8E7K3CCV9', 'A14CCWIFLDPZLA', 'ABDXLB3YCM6XY', 'A27JUKQFY00XP7', 'A29HILIOBLRW1S', 'A30LVINW75GV0V', 'A26EEQE7NYLQNI', 'A3A6BVDDO2M1C9', 'A2E6Z6NDAES26P', 'A2J9NFY45NKR2N', 'A1MDKN4ZIZYH2O', 'AMOBC1D09Q381', 'A3MHMMTUTOJQ1V', 'A1HEB7F1J8ENZI', 'A3ZSLY8UQDRC3', 'A2LBYE8RUQBQYG', 'A1LJRMT897VBKP', 'AFA73M8WJKEN0', 'ARPBDM5QZ4XQC', 'AV6C736XAEG67', 'A3EM9OBO5KU9GG', 'A2ZT2Z0LQ05EGZ'}
39


In [103]:
a1 = data1.groupby(['Input.url']).label.nunique().eq(1).value_counts()
a2 = data2.groupby(['Input.url']).label.nunique().eq(1).value_counts()
print(a1)
print(a1[1]/(n_data_points/3))
print(a2)
print(a2[1]/(n_data_points/3))



False    137
True      78
Name: label, dtype: int64
0.3611111111111111
False    119
True      96
Name: label, dtype: int64
0.4444444444444444


In [104]:
data1_unan_urls = []
data2_unan_urls = []

for url in data1['Input.url'].unique():
    if data1.groupby(['Input.url']).label.nunique().eq(1).loc[url] == True:
        data1_unan_urls.append(url)
    if data2.groupby(['Input.url']).label.nunique().eq(1).loc[url] == True:
        data2_unan_urls.append(url)

data1_unanimous = data1[data1['Input.url'].isin(data1_unan_urls)]
# print(len(data1_unanimous))
data2_unanimous = data2[data2['Input.url'].isin(data2_unan_urls)]
# print(len(data2_unanimous))

a3 = data1_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
a4 = data2_unanimous.groupby(['Input.url']).label.nunique().eq(1).value_counts()
print("a3")
print(a3)

print("a4")
print(a4)

# for url in data1['Input.url'].unique():
#     if data1.loc[data1['Input.url']==url, 'label'].value_counts().iloc[0] != data2.loc[data2['Input.url']==url, 'label'].value_counts().iloc[0]:
#         print(url)
#         print(data1.loc[data1['Input.url']==url, 'label'].value_counts())
#         print(data2.loc[data2['Input.url']==url, 'label'].value_counts())
#         print('\n')

# for url in data1['Input.url'].unique():
#     if data1.loc[data1['Input.url']==url, 'label'].value_counts().iloc[0] != data2.loc[data2['Input.url']==url, 'label'].value_counts().iloc[0]:
#         print(url)
#         print(data1.loc[data1['Input.url']==url, 'label'].value_counts())
#         print(data2.loc[data2['Input.url']==url, 'label'].value_counts())
#         print('\n')

# for i in data1.iterrows():
#     if i[1]['label'] != i[1]['Input.m_1']:
#         print(i)
#         break

# data1_unan = data1[["Input.url","label","Input.m_1"]]
# data2_unan = data2[["Input.url","label","Input.m_1"]]

# data1_unan = data1_unan[]


# data1_unan = data1[data1.groupby(['Input.url']).label.nunique().eq(1)]
# data2_unan = data2[data2.groupby(['Input.url']).label.nunique().eq(1)]


a3
True    78
Name: label, dtype: int64
a4
True    96
Name: label, dtype: int64


In [105]:

data1_unanimous['correct_label'] = (data1_unanimous['label']==data1_unanimous['Input.m_1'])
print(len(data1_unanimous))
a = data1_unanimous['correct_label'].value_counts()
print(a)
print(a[1]/len(data1_unanimous))

data2_unanimous['correct_label'] = (data2_unanimous['label']==data2_unanimous['Input.m_1'])
print(len(data2_unanimous))
b = data2_unanimous['correct_label'].value_counts()
print(b)


print(b[1]/len(data2_unanimous))

234
True     165
False     69
Name: correct_label, dtype: int64
0.7051282051282052
288
True     174
False    114
Name: correct_label, dtype: int64
0.6041666666666666


<ipython-input-105-7516a29b1b9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_unanimous['correct_label'] = (data1_unanimous['label']==data1_unanimous['Input.m_1'])
<ipython-input-105-7516a29b1b9b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2_unanimous['correct_label'] = (data2_unanimous['label']==data2_unanimous['Input.m_1'])


In [ ]:
data1_unanimous_and_confident['correct_label'] = (data1_unanimous['label']==data1_unanimous['Input.m_1'])
print(len(data1_unanimous))
a = data1_unanimous['correct_label'].value_counts()
print(a)
print(a[1]/len(data1_unanimous))

data2_unanimous['correct_label'] = (data2_unanimous['label']==data2_unanimous['Input.m_1'])
print(len(data2_unanimous))
b = data2_unanimous['correct_label'].value_counts()
print(b)


print(b[1]/len(data2_unanimous))

In [29]:
data1.groupby(['Input.url','WorkerId'])
data2.groupby(['Input.url','WorkerId'])

In [ ]:
df.groupby(['id', 'group', 'term']).size().unstack(fill_value=0)